In [62]:
import pickle

# 1121An arbitrary collection of objects supported by pickle.
data = {
    'a': [1, 2.0, 3, 4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    

In [63]:
import pickle

with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

In [64]:
data['a']

[1, 2.0, 3, (4+6j)]

In [1]:
import MySQLdb
import gdal
from osgeo import ogr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy
def connectdb():
    print('连接到mysql服务器...')
    # 打开数据库连接
    # 用户名:hp, 密码:Hp12345.,用户名和密码需要改成你自己的mysql用户名和密码，并且要创建数据库TESTDB，并在TESTDB数据库中创建好表Student
    db = MySQLdb.connect("localhost","root","123456","bigdata",charset="utf8")
    print('连接上了!')
    return db
db = connectdb()   
cursor = db.cursor()
sqlin = "Select * from stationin"
sqlout="Select * from stationout"

dfin = pd.read_sql(sqlin, db)
dfout= pd.read_sql(sqlout, db)
dfin=dfin.set_index('startTime')
dfout=dfout.set_index('startTime')

连接到mysql服务器...
连接上了!


In [2]:
import datetime
###in data producer
threshold=60*60 ### one hour threshold
delta=datetime.timedelta(minutes=5)
h=int(threshold/delta.seconds)
allPre_vectorin=[]
for i in range(len(dfin)):
    starttime_str=dfin.index[i]
    datavector=dfin.iloc[i][:]
    starttime_stamp= datetime.datetime.strptime(starttime_str, '%Y-%m-%d %H:%M:%S')
    pre_vector=[]
    for j in range(h):
        preTime_stamp=starttime_stamp-delta*(j+1)
        pretime_str=preTime_stamp.strftime('%Y-%m-%d %H:%M:%S')
        try:
            pre_vector.append(list(dfin.loc[pretime_str][0:]))
        except:
            pre_vector.append(list(np.zeros(dfin.shape[1])))
    allPre_vectorin.append(pre_vector)

In [3]:
### out data producer
allPre_vectorout=[]
for i in range(len(dfout)):
    starttime_str=dfout.index[i]
    datavector=dfout.iloc[i][:]
    starttime_stamp= datetime.datetime.strptime(starttime_str, '%Y-%m-%d %H:%M:%S')
    pre_vector=[]
    for j in range(h):
        preTime_stamp=starttime_stamp-delta*(j+1)
        pretime_str=preTime_stamp.strftime('%Y-%m-%d %H:%M:%S')
        try:
            pre_vector.append(list(dfout.loc[pretime_str][0:]))
        except:
            pre_vector.append(list(np.zeros(dfout.shape[1])))
    allPre_vectorout.append(pre_vector)

In [5]:

### store in pickle
import pickle
data1 = {
    'label_in': dfin, ##dataframe
    'label_out': dfout,
    'data_in': allPre_vectorin, ##array类型
    'data_out': allPre_vectorout,
    'aver_in': vectorin_sametimeslot,
    'aver_out': vectorout_sametimeslot
}
with open('data1.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data2 = pickle.load(f)

In [5]:
###计算协方差矩阵
#allPre_vectorin=data2['data_in']
#allPre_vectorout=data2['data_out']
allPre_vectorout=np.array(allPre_vectorout)
allPre_vectorin=np.array(allPre_vectorin)
CovPre_vectorin=[]
CovPre_vectorout=[]
for i in range(allPre_vectorin.shape[0]):
    CovPre_vectorin.append(np.cov(np.array(allPre_vectorin)[i].T).tolist())
    CovPre_vectorout.append(np.cov(np.array(allPre_vectorout)[i].T).tolist())
CovPre_vectorin=np.array(CovPre_vectorin)
CovPre_vectorout=np.array(CovPre_vectorout)

In [140]:
vectorin_sametimeslot=[]
vectorout_sametimeslot=[]
for i in range(dfin.shape[0]):
    currentvector=dfin.iloc[i]
    vectorin=[]
    for j in range(1,7):
        othervector=dfin.iloc[(i+j*216)%(216*7)]
        othertimestamp=datetime.datetime.strptime(dfin.index[(i+j*216)%(216*7)], '%Y-%m-%d %H:%M:%S')
        if othertimestamp.day!=11 and othertimestamp.day!=12:
            #print(othertimestamp.day)
            vectorin.append(list(othervector))
#         else:
#             vectorin.append(list(np.zeros(dfin.shape[1]))
    #print(len(vectorin),i)
    vectorin_sametimeslot.append(vectorin[0:4])
    
for i in range(dfout.shape[0]):
    currentvector=dfout.iloc[i]
    vectorout=[]
    for j in range(1,7):
        othervector=dfout.iloc[(i+j*216)%(216*7)]
        othertimestamp=datetime.datetime.strptime(dfout.index[(i+j*216)%(216*7)], '%Y-%m-%d %H:%M:%S')
        if othertimestamp.day!=11 and othertimestamp.day!=12:
            vectorout.append(list(othervector))
#         else:
#             vectorout.append(list(np.zeros(dfin.shape[1])))
    vectorout_sametimeslot.append(vectorout[0:4])
    
vectorin_sametimeslot=np.array(vectorin_sametimeslot)
vectorout_sametimeslot=np.array(vectorout_sametimeslot)

In [144]:
data2['aver_out']

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  1.,  0., ...,  1.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  1.,  0.,  0.]],

       ...,

       [[ 0.,  0.,  3., ...,  1., 37.,  0.],
        [ 0.,  0.,  6., ...,  1., 29.,  0.],
        [ 0.,  0.,  8., ...,  0., 35.,  0.],
        [ 0.,  0.,  7., ...,  0., 29.,  0.]],

       [[ 0.,  0.,  0., ...,  0., 20.,  0.],
        [ 0.,  0.,  0., ...,  0.,  2.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0., 19.,  0.],
        [ 0.,  0.,  0., ...,  0.

In [42]:
dfin.index[(200+216*3)%(216*7)],dfin.index[200]

('2017-02-13 22:40:00', '2017-02-10 22:40:00')